# DocumentReference factory

create a bunch of DopcumentReference objects from a csv file and save them to a folder and create Bundle objects from the conditions and save them to a folder

1. Load DocumentReference Template
2. get csv in form

|DocumentReference #|_id|_datetime|_code|_file|
|---|---|---|---|---|
|Foo|foo|2023-08-21T20:16:25.019Z|12345678|Examplitis|

4. Convert csv to DataFrame
<!-- 2. Create a transaction Bundle
1. Load Patient into Bundle
2. Load the data row into the DocumentReference template
3. Add to the Bundler -->
5. Create DocumentReference Instance
5. Validate 

### Define Globals and Imports

In [21]:
from hashlib import sha1
import base64
from pathlib import Path
from IPython.display import display, Markdown as md, HTML
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources.documentreference import DocumentReference
from json import dumps, loads
import pandas as pd
from requests import get, post
from datetime import datetime
from dateutil import tz

in_path = Path.cwd() / 'source_data' /'docref.csv'
out_path = Path.cwd() /'examples'
print(in_path)
print(out_path)
# _system = 'http://snomed.info/sct'
_system = 'http://loinc.org'
fhir_test_server = 'https://r4.ontoserver.csiro.au/fhir'

_now = datetime.now(tz=tz.tzlocal()).isoformat()
print(_now)

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json',
}
# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient


/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/source_data/docref.csv
/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/resource_maker/examples
2023-08-23T19:06:42.450492-07:00


In [22]:
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/PatientHealthQuestionnaire-2_v1.0_2014Jul2.pdf")
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/AHCM-HealthSocialNeedsScreeningTool-rev-8_10_2021.pdf")
# my_path = Path(r"/Users/ehaas/Documents/FHIR/SAIG/input/images/Example3PRAPARE-English.pdf")

### Load Docref Template

In [23]:
# title ='Example3PRAPAREQuestionnaire' #'Example2AHN-HRSNQuestionnaire' #'Example1PHQ-2Questionnaire'
docref  = DocumentReference.parse_file('/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/QR-OBs-DE/docref.yml')
docref.id

'docref'

###  Get CSV File ( hard coded for now) and Convert to DF

In [24]:
from io import StringIO  # for hardcoded csv only
my_csv = '''_title,_id,_datetime,_code,_file

Progress Note,progress-note,2023-08-21T20:16:25.019Z,11506-3,/Users/ehaas/Downloads/progress-note.pdf
Imaging Study,imaging-study,2021-01-21,18748-4,/Users/ehaas/Downloads/imaging-study.pdf
Surgical Note,surgical-note,2018-02-02,11504-8,/Users/ehaas/Downloads/surgical-note.pdf
CCD,ccd,2023-08-22,34133-9,/Users/ehaas/Downloads/example-1-ccd.xml
'''

df = pd.read_csv(StringIO(my_csv), sep=",", dtype='str')

df

,_title,_id,_datetime,_code,_file
0,Progress Note,progress-note,2023-08-21T20:16:25.019Z,11506-3,/Users/ehaas/Downloads/progress-note.pdf
1,Imaging Study,imaging-study,2021-01-21,18748-4,/Users/ehaas/Downloads/imaging-study.pdf
2,Surgical Note,surgical-note,2018-02-02,11504-8,/Users/ehaas/Downloads/surgical-note.pdf
3,CCD,ccd,2023-08-22,34133-9,/Users/ehaas/Downloads/example-1-ccd.xml


### Create Condition Instance

1. Get Data Element row by row from DF

In [25]:
def get_display(_system,_code,_version=None):
    """for example Looking up a code/system
       GET [base]/CodeSystem/$lookup?system=http://loinc.org&code=1963-8
    """
    params = dict(
       system = _system,
      # system = 'http://loinc.org',
      code = _code,
      # code = '1963-8'
        )
    r = get(f'{fhir_test_server}/CodeSystem/$lookup', params = params, headers = headers)
    print(r.status_code, r.reason)
    print(r.json())
    if r.status_code != 200:
        return 'examplitis'
    else:
      return r.json()['parameter'][1]['valueString']

In [26]:
def get_binary(file_path):
    """get binary from file path"""
    with open(file_path, 'rb') as f:
        binary = base64.b64encode(f.read()).decode('utf-8')
    return binary

In [27]:
for i in range(len(df)):
    # print(i, df.iloc[i])
    docref.meta.extension = None  #remove title and description
    # docref.meta.extension[0].valueString = f'DocRef {title}'
    # docref.meta.extension[1].valueMarkdown = f'This is a an example of {title}  \
    # for the *US Core DocumentReference Profile*'
    _title,_id,_datetime,_code,_file = df.iloc[i]
    _display = get_display(_system,_code)
    docref.id = _id
    docref.date = _now
    docref.type.coding[0].code = _code
    docref.type.coding[0].display = _display
    docref.type.text = _display.title()
    docref.content[0].attachment.title = _title
    docref.content[0].attachment.contentType = "application/pdf" if _file.endswith('pdf') else "text/xml"
    docref.content[0].attachment.creation = _datetime
    docref.content[0].attachment.data = get_binary(_file)
    docref.content[0].attachment.creation = _datetime
    out_file = out_path / f'DocumentReference-{_title}.json'
    out_file.write_text(docref.json(indent=2))

200 OK
{'resourceType': 'Parameters', 'parameter': [{'name': 'code', 'valueCode': '11506-3'}, {'name': 'display', 'valueString': 'Progress note'}, {'name': 'name', 'valueString': 'LOINC v2.74'}, {'name': 'system', 'valueUri': 'http://loinc.org'}, {'name': 'version', 'valueString': '2.74'}, {'name': 'property', 'part': [{'name': 'code', 'valueCode': 'parent'}, {'name': 'value', 'valueCode': 'LP416685-8'}]}, {'name': 'property', 'part': [{'name': 'code', 'valueCode': 'inactive'}, {'name': 'value', 'valueBoolean': False}, {'name': 'valueBoolean', 'valueBoolean': False}]}, {'name': 'designation', 'part': [{'name': 'language', 'valueCode': 'es-MX'}, {'name': 'use', 'valueCoding': {'system': 'http://terminology.hl7.org/CodeSystem/hl7TermMaintInfra', 'code': 'preferredForLanguage', 'display': 'Preferred For Language'}}, {'name': 'value', 'valueString': 'Nota de progreso:{Configuración} :Punto temporal:Tipo:Documento:{Role}'}]}, {'name': 'designation', 'part': [{'name': 'language', 'valueCode'